# Build a Delft-FIAT model for the FloodAdapt backend

This notebook demonstrates how to set up a Delft-FIAT model anywhere in the world ready to be used in FloodAdapt. As an example we will create a model for the city of Kingston upon Hull, UK, in the Humber delta. The relevant data will be taken from [OpenStreetMap (OSM)](https://www.openstreetmap.org) and the [global flood vulnerability datasets from JRC](https://publications.jrc.ec.europa.eu/repository/handle/JRC105688). Both the data sources and the region of interest (the model domain) can be changed according to the user's wishes.

*Disclaimer: The outcomes of this model are not validated*

## **Step 0**: Import required packages
First we need to import the necessary python packages.

In [1]:
from hydromt_fiat.fiat import FiatModel
from hydromt.log import setuplog
from hydromt.data_catalog import DataCatalog
from pathlib import Path
import geopandas as gpd

## **Step 1:** List Input Names
Here we list the input paths and names of some needed utilities:
- `model_name`: This is the name for the overall FloodAdapt folder where everything will be stored. Named after region of interest (in this case)
- `model_path`: The full path of the FloodAdapt model folder
- `fiat_root`: Path to folder parsed by FloodAdapt where FIAT model is stored.
- `fiat_logger_name`: Name for the FIAT logger
- `region_fn`: Path to geojson file of the region of interest. Used when building FIAT domain
- `data_catalog_fn`: Path to data_catalog yml file

In [2]:
model_name = 'Humber'
model_path = Path('/home/wotromp/InterTwin/FloodAdapt_database') / model_name

fiat_root = model_path / Path('static/templates/tmp2/fiat')

fiat_logger_name = "FIAT_logger"

region_fn = model_path / 'HumberDelta_large.geojson'

data_catalog_fn = Path('/home/wotromp/InterTwin/Data/Humber/data_catalog.yml')

## **Step 2**: Initialize
In this step we initialize HydroMT-FIAT with the `model_folder`, `data_catalog`, and `logger` that we defined above.

In [ ]:
fiat_logger = setuplog(fiat_logger_name, log_level=10)

fiat = FiatModel(root=fiat_root, data_libs=[data_catalog_fn], mode='w+', logger=fiat_logger)

## **Step 3a:** Configure model - Setup Region
Build the model domain with the geojson file specified earlier. We also need to indicate the country and continent the model domain is in.

In [4]:
region = gpd.read_file(region_fn)

continent = 'Europe'
country = 'United Kingdom'

To speed up the configuration of the aggregation area we first create a cutout of our model domain and save it for later when we use it when building the model

In [5]:
dc = DataCatalog(data_libs=data_catalog_fn)

agg_areas = dc.get_geodataframe('gadm_level3', geom=region)
if not (model_path/"static"/"aggregation_areas").exists():
    (model_path/"static"/"aggregation_areas").mkdir()
agg_areas.to_file(model_path/"static"/"aggregation_areas"/"agg_areas.gpkg", driver='GPKG')

## **Step 3b:** Configure model - Configure input data

Next, we need to specify properties of the various dataset. For the vulnerability and exposure data HydroMT-FIAT needs the names and unit of the datasets. The names should correspond to the names (keys) in the data catalog except from the OSM data, which is accessed with the [OSMnx Python package](https://geoffboeing.com/publications/osmnx-complex-street-networks/). Should you wish to use different data sources, *make sure to update the data catalog*. For the output data, we need to specify the output file names and where to store them. The parameter names below are parsed by the model builder, so they should not be changed.

**Vulnerability**
- `vulnerability_fn`: the source name of the vulnerability curve dataset as defined in the HydroMT-FIAT global data catalog. In this example, we use the JRC global damage curves.
- `vulnerability_identifiers_and_linking_fn`: the source name of the *occupancy type-vulnerability curves* linking table as defined in the HydroMT-FIAT data catalog. In this example, we use the default table for linking the OSM land use classese to the JRC damage curves (i.e., the assets classified as residential links to the residential vulnerability curves of JRC).
- `unit`: the unit of the vulnerability curves. The JRC curves are in meters.

**Exposure**
- `asset_locations`: the source name of the location and (optionally) geometry data of the assests for which damages will be calculated. In this example, the building footprints from OSM are used.
- `occupancy_type`: the source name of the occupancy type data to classify the assets. In this example, the land use data from OSM is used.
- `max_potential_damage`: the source name of the maximum potential damage values data. In this example, the JRC maximum damage values are used.
- `ground_floor_height`: the height of the ground floor of all assets, in the same `unit`
- `unit`: the unit of the exposure data

**Output**
- `output_dir`: the name of the output directory
- `output_csv_name`: the name of the output CSV
- `output_vector_name`: the name of the vector output file(s)

*Note: The unit names are parsed by FloodAdapt. See FloodAdapt's UnitfulValue for correct formatting*

In [6]:
### Setup vulnerability parameters ###
vulnerability_fn = "jrc_vulnerability_curves"
vulnerability_identifiers_and_linking_fn = "jrc_vulnerability_curves_linking"
unit = "meters"

### Setup exposure parameters ###
asset_locations = "OSM"
occupancy_type = "OSM"
max_potential_damage = "jrc_damage_values"
ground_floor_height = 0
ground_floor_height_unit = "meters"
damage_types = ['total']
ground_elevation_file = model_path / 'static' / 'dem' / "Humber_subgrid.tif"

### Setup output parameters ###
output_dir = "output"
output_csv_name = "output.csv"
output_vector_name = "spatial.gpkg"

### Setup Aggregation Zone parameters ###
aggregation_area_fn = model_path/"static"/"aggregation_areas"/"agg_areas.gpkg"
attribute_names = "NAME_3"
label_names = "AdminZonesLevel3"
file_names = "gadm_level3"

In [7]:
configuration = {
    "setup_output": {
        "output_dir": output_dir,
        "output_csv_name": output_csv_name,
        "output_vector_name": output_vector_name,
    },
    "setup_vulnerability": {
        "vulnerability_fn": vulnerability_fn,
        "vulnerability_identifiers_and_linking_fn": vulnerability_identifiers_and_linking_fn,
        "continent": continent,
        "unit": unit,
    },
    "setup_exposure_buildings": {
        "asset_locations": asset_locations,
        "occupancy_type": occupancy_type,
        "max_potential_damage": max_potential_damage,
        "ground_floor_height": ground_floor_height,
        "unit": ground_floor_height_unit,
        "country": country,
        "damage_types": damage_types,
        "ground_elevation_file": ground_elevation_file,
    },
    "setup_aggregation_areas": {
        "aggregation_area_fn": aggregation_area_fn,
        "attribute_names": attribute_names,
        "label_names": label_names,
        "file_names": file_names,
    },
}

## **Step 4:** Build model

In this step we build the Delft-FIAT model. Depending on the extent of the model it can take some time to download the building footprints and land use data from OSM. During the building of the model, log messages display what is happening.

*Note that the model will not yet be written because of setting the write argument to False.*

In [ ]:
fiat.build(region={"geom": region}, opt=configuration, write=True)

## **Step 5**: Inspect model
We now inspect the resulting exposure data and vulnerability curves that will be saved in the `fiat` instance.

### Exposure data
In the map below, the region and Secondary Object Type of the exposure objects are plotted. You can zoom in and see whether the data makes sense, perhaps using Google maps and/or streetview to validate the occupancy types.

*Note: In Delft-FIAT, exposure is defined with object footprints, lines or centroids. In this example we are only using the buildings extracted from the OSM data. This means we are not going to look into, e.g., farmland, roads, etc. but that is possible with Delft-FIAT.*

In [ ]:
# This visualization cell is quite computationally heavy. Skip for now.
# Get the geodataframe with exposure data
# gdf = fiat.exposure.get_full_gdf(fiat.exposure.exposure_db)

# Plot the region and the secondary object types of the exposure data
# m = region.explore(name='Region', style_kwds={'color': 'black', 'fill': False})
# m = gdf.explore(m=m, column='Primary Object Type', name='Exposure types')
# m

## **Step 6:** Write model

Finally, write the model in a folder structure appropriate for FloodAdapt. The right folders should be created automatically if our `model_path` and `fiat_root` are correct.

In [ ]:
fiat.write()

def tree(directory):
    print(f"+ {directory}")
    for path in sorted(directory.rglob('*')):
        depth = len(path.relative_to(directory).parts)
        spacer = "  " * depth
        print(f"{spacer}+ {path.name}")


tree(Path(fiat.root))